In [2]:
import math

import numpy as np
from scipy import signal
import utils
import mediapipe as mp
import cv2
from scipy.spatial import ConvexHull

In [4]:
def generateMaskFromPoints(ROI, face, w, h):
    points = []
    for current_point in ROI:
        x = int(face.landmark[current_point].x * w)
        y = int(face.landmark[current_point].y * h)
        points.append([x, y])

    mask = np.zeros((h, w), dtype=np.uint8)
    points = np.array(points, dtype=np.int32)
    cv2.fillConvexPoly(mask, points, 255)

    # suavizar bordes
    mask = cv2.GaussianBlur(mask, (25, 25), 0)

    return mask

In [ ]:
mp_face_mesh = mp.solutions.face_mesh

video_path = "Dataset/Patient_1/Q1_1/vid_crop.avi"

cap = cv2.VideoCapture(video_path)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Generate the skin mask
    #########################3
    h, w, _ = frame.shape
    with mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5) as face_mesh:

        results = face_mesh.process(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
        if not results.multi_face_landmarks:
            print("Error A")
            break

        face = results.multi_face_landmarks[0]
    
        ROIS = [ 
            # FRENTE
            [104, 103, 67, 109, 10, 338, 297, 332, 333, 299, 337, 151, 108, 69]
            #MEJILLAS IZQ / DER
            #,[ 206, 216, 214, 192, 147, 123, 117, 118, 101, 36]
            
        ]
        
        final_mask = np.zeros(h, w)
        for ROI in ROIS:
            current_mask = generateMaskFromPoints(ROI, face, w, h)

    # Make a copy and black out all pixels outside the mask
    masked_frame = np.zeros_like(frame_rgb)
    masked_frame[mask > 0] = frame_rgb[mask > 0]

    # Show
    cv2.imshow("Skin Mask Only", cv2.cvtColor(masked_frame, cv2.COLOR_RGB2BGR))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


KeyboardInterrupt: 